# ECM calculation in several molecules

**Task:** 

1. Define the molecules that will be under study as well as their associated defining vectors (from https://csm.ouproj.org.il/molecule)
2. Import the CFMAR molecule from a xyz file
3. Align the molecule so that the vector lies on the z-axis
4. Calculate and print CCMs and ECMs
5. Save data and generate Excel files

In [1]:
# creating the path (PYTHONPATH) to our module.
# assuming that our 'pyECM' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
from pyECM.molecule_class import molecula

N_molecules = 2
names_molecules = [None]*N_molecules
vectores = [None]*N_molecules

vectores[0]= np.array([-0.0172, 0.0068, 0.9998])
names_molecules[0] = 'CFMAR_chiral'

vectores[1]= np.array([-0.0083, 0.0045, 1.0000])
names_molecules[1] = 'AP1_chiral'

In [3]:
import pandas as pd

#Define basis set
base='sto-3g'

#Define lists
zrate = np.zeros([N_molecules])
Norms1 = np.zeros([N_molecules])
CCMs1 = np.zeros([N_molecules])
Norms2 = np.zeros([N_molecules])
CCMs2 = np.zeros([N_molecules])
ECMs_NR = np.zeros([N_molecules])
ECMs_4c = np.zeros([N_molecules])

# Get and save CCM and ECM values (takes about 2 min)
for i in range(N_molecules):
    mymolecule = molecula(XYZ_file = '../pyECM/data/import/'+ names_molecules[i] +'.xyz', direction=vectores[i])
    mymolecule.rotate_to_align_with_z()
    mymolecule.export_xyz(folder='../pyECM/data/export/', prefix_name=names_molecules[i], DIRAC = False)

    #Norms1[i], CCMs1[i], Norms2[i], CCMs2[i] = mymolecule.CCM()
    #ECMs_NR[i], ECMs_molcontr, ECMs_4c[i] = mymolecule.ECM(name='../pyECM/data/export/'+names_molecules[i], tracking=False, basis_set = base, fourcomp=False, debug=0)
    
    mymolecule.CCM()
    Norms1[i], CCMs1[i], Norms2[i], CCMs2[i] = mymolecule.Norm1, mymolecule.CCM1, mymolecule.Norm2, mymolecule.CCM2

    
    #Define options for the Gaussian Type Orbitals (generated by pySCF) and the WF calculation.
    GTO={'basis' : base, 'charge' : 0, 'spin' : 0}
    WF_method = {'fourcomp': False, 'debug' : 0}
    
    # Get the WF with the pySCF code
    mymolecule.pySCF_WF(name='../pyECM/data/export/'+ names_molecules[i], gto_dict=GTO, method_dict=WF_method)

    #Calculate ECM
    method = {'fourcomp': False, 'debug' : 0}
    mymolecule.ECM(name='../pyECM/data/export/'+ names_molecules[i], method_dict=method)

    ECMs_NR[i], ECMs_4c[i] = mymolecule.ECM_NR, mymolecule.ECM_4c
    
    
    matrix_aux = np.vstack([Norms1[i], CCMs1[i], Norms2[i], CCMs2[i], ECMs_NR[i], ECMs_4c[i]])
    matrix     = np.transpose(matrix_aux)

    #Need some modification.
    #matrix_aux2 = np.vstack([ECMs_molcontr])
    #matrix_aux2 = np.transpose(matrix_aux2)
    
    #np.save('../pyECM/data/results/'+names_molecules[i]+'_molcontr',matrix_aux2)
    np.save('../pyECM/data/results/'+names_molecules[i],matrix)


In [4]:
#Create Tables
import pandas as pd

dataframes = []
dataframes_original_mols = []
dataframes_perorbital = []

for i in range(N_molecules):
    #Import CCM & ECM obtained results 
    dataframes.append(pd.DataFrame(np.load('../pyECM/data/results/'+names_molecules[i]+'.npy')))
    dataframes[i].columns = ['Norms 1', 'CCM 1', 'Norms 2', 'CCM 2', 'ECM_NR', 'ECM_4c']

    #Import ECM per orbital results
    #dataframes_perorbital.append(pd.DataFrame(np.load('../pyECM/data/results/'+names_molecules[i]+'_molcontr.npy')))


#Export total ECM results to xlsx file (one sheet by molecule)
with pd.ExcelWriter('Results.xlsx') as writer:
    for indice in range(N_molecules):
        dataframes[indice].to_excel(writer,sheet_name=names_molecules[indice], index=False, startrow=0, header=True)

#Export ECM per molecular orbital to xlsx file (one sheet by molecule)
#with pd.ExcelWriter('Analysis_per_orbital.xlsx') as writer:
#    for indice in range(N_molecules):
#        dataframes_perorbital[indice].to_excel(writer,sheet_name=names_molecules[indice], index=False, header=False)